In [1]:
pip install wandb

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install gym

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install imageio

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install ale-py


Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install gym[accept-rom-license]

Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
import tensorflow as tf

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # 0 = all messages, 1 = filter out INFO, 2 = filter out WARNING, 3 = filter out ERROR
tf.get_logger().setLevel('ERROR')

import warnings
warnings.filterwarnings('ignore')


import logging
gym_logger = logging.getLogger('gym')
gym_logger.setLevel(logging.ERROR)

2023-12-11 18:42:31.554581: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-11 18:42:31.592558: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-11 18:42:31.592598: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-11 18:42:31.593874: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-11 18:42:31.600017: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-11 18:42:31.600971: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [ ]:
#API KEY: 73c9a156b91f3e0c01c3d5f332d23bfc66f4cdbf

In [2]:
import wandb
wandb.init(project="ActorCritic", entity = "rl_proj")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: 1630423 (rl_proj). Use `wandb login --relogin` to force relogin


In [ ]:
import os
import gym
import random
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from collections import namedtuple, deque
from ale_py import ALEInterface
import wandb
import imageio
import matplotlib.pyplot as plt

# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

ale = ALEInterface()

class Config:
    EPSILON_START = 1.0
    EPSILON_END = 0.01
    EPSILON_DECAY_RATE = 0.99
    EPISODES = 1500  
    BATCH_SIZE = 128
    GAMMA = 0.999
    MAX_STEPS_PER_EPISODE = 1000
    LEARNING_RATE = 1e-4 
    MEMORY_SIZE = 10000

config = Config()
best_reward = 0

env = gym.make("Assault-v4", render_mode="rgb_array")
n_actions = env.action_space.n

class ReplayMemory(object):
    def __init__(self, capacity):
        self.memory = deque([], maxlen=capacity)
        self.transition = namedtuple('Transition', ('state', 'action', 'next_state', 'reward', 'done'))

    def push(self, *args):
        self.memory.append(self.transition(*args))

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)



class Actor(keras.Model):
    def __init__(self, n_actions):
        super(Actor, self).__init__()
        #self.conv1 = layers.Conv2D(32, 8, strides=4, activation="relu")
        self.conv1 = layers.Conv2D(32, 8, strides=4, activation="relu", kernel_initializer='he_normal')
        #self.conv2 = layers.Conv2D(64, 4, strides=2, activation="relu")
        self.conv2 = layers.Conv2D(64, 4, strides=2, activation=None)  # Remove activation here
        self.batch_norm1 = layers.BatchNormalization()
        self.conv3 = layers.Conv2D(64, 3, strides=1, activation="relu")
        self.flatten = layers.Flatten()
        self.d1 = layers.Dense(512, activation="relu", kernel_regularizer=keras.regularizers.l2(0.001))
        self.d2 = layers.Dense(n_actions, activation="softmax")  # Output layer for action probabilities

    def call(self, inputs):
        x = self.conv1(inputs)
        #x = self.conv2(x)
        x = self.conv2(x)
        x = tf.nn.relu(self.batch_norm1(x))
        x = self.conv3(x)
        x = self.flatten(x)
        x = self.d1(x)
        return self.d2(x)


class Critic(keras.Model):
    def __init__(self):
        super(Critic, self).__init__()
        self.conv1 = layers.Conv2D(32, 8, strides=4, activation="relu")
        self.conv2 = layers.Conv2D(64, 4, strides=2, activation="relu")
        self.conv3 = layers.Conv2D(64, 3, strides=1, activation="relu")
        self.flatten = layers.Flatten()
        self.d1 = layers.Dense(512, activation="relu")
        self.d2 = layers.Dense(1)

    def call(self, inputs):
        x = self.conv1(inputs)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.flatten(x)
        x = self.d1(x)
        return self.d2(x)


actor_model = Actor(n_actions)
print("Number of actions:", n_actions)
actor_model.build(input_shape=(None, 210, 160, 3))  # Build the model with the input shape
actor_model.summary()
critic_model = Critic()


lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-4,
    decay_steps=10000,
    decay_rate=0.9)
actor_optimizer = keras.optimizers.Adam(learning_rate=lr_schedule)
critic_optimizer = keras.optimizers.Adam(learning_rate=lr_schedule)

memory = ReplayMemory(config.MEMORY_SIZE)


def take_action(state, epsilon):

    if random.random() < epsilon:
        return env.action_space.sample()
    else:
        action_probabilities = actor_model.predict(state[np.newaxis, ...])
        print("Action probabilities:", action_probabilities)

        if np.isnan(action_probabilities).any():
            print("NaN detected in action probabilities")
            return env.action_space.sample() 
        return np.random.choice(n_actions, p=np.squeeze(action_probabilities))

def optimize_model():
    if len(memory) < config.BATCH_SIZE:
        return  # Exit the function if not enough samples

    # Sample a batch of transitions from the replay memory
    transitions = memory.sample(config.BATCH_SIZE)
    batch = memory.transition(*zip(*transitions))

    # Convert the batches into numpy arrays for processing
    state_batch = np.array(batch.state)
    action_batch = np.array(batch.action)
    reward_batch = np.array(batch.reward)
    next_state_batch = np.array(batch.next_state)
    done_batch = np.array(batch.done)

    # Critic Update
    with tf.GradientTape() as tape:
        # Get the values from the Critic model
        values = critic_model(state_batch)
        # Create a dummy target for simplicity
        dummy_target = tf.random.uniform(shape=values.shape)
        # Compute a simple mean squared error

        values_squeezed = tf.squeeze(values)
        if len(values_squeezed.shape) > 1:
            raise ValueError("Critic model's output is not a 1D array")

        critic_loss = tf.math.reduce_mean(tf.math.square(dummy_target - values))

    critic_grads = tape.gradient(critic_loss, critic_model.trainable_variables)
    critic_grads, _ = tf.clip_by_global_norm(critic_grads, 1.0)  # Gradient clipping
    critic_optimizer.apply_gradients(zip(critic_grads, critic_model.trainable_variables))

    # Debugging: Print gradients and corresponding variables
    for grad, var in zip(critic_grads, critic_model.trainable_variables):
        if grad is None:
            print(f"Gradient is None for variable {var.name}")

    # Filter out None gradients
    critic_grads_and_vars = [(grad, var) for grad, var in zip(critic_grads, critic_model.trainable_variables) if grad is not None]

    # Apply gradients if there are valid ones
    if critic_grads_and_vars:
        critic_optimizer.apply_gradients(critic_grads_and_vars)
    else:
        print("No valid gradients to apply.")

    # Actor Update
    with tf.GradientTape() as tape:
        # Predict the action probabilities for the current state
        #print("Input shape to the model:", state_batch.shape)
        action_probs = actor_model(state_batch)
        # Create a one-hot encoded mask for the taken actions
        action_mask = tf.one_hot(action_batch, n_actions)
        # Select the probabilities for the actions that were actually taken
        selected_action_probs = tf.reduce_sum(action_probs * action_mask, axis=1)

        # Adjust dummy target values shape to match the values
        dummy_target_values = np.zeros_like(values_squeezed.numpy())
        advantage = dummy_target_values - values_squeezed

        epsilon = 1e-8
        actor_loss = -tf.math.reduce_mean(tf.math.log(selected_action_probs + epsilon) * advantage)
        #actor_loss = -tf.math.reduce_mean(tf.math.log(selected_action_probs) * advantage)

    actor_grads = tape.gradient(actor_loss, actor_model.trainable_variables)
    actor_grads, _ = tf.clip_by_global_norm(actor_grads, 0.5)  # Gradient clipping
    actor_optimizer.apply_gradients(zip(actor_grads, actor_model.trainable_variables))
    for grad in actor_grads:
        if tf.reduce_any(tf.math.is_inf(grad)).numpy() or tf.reduce_any(tf.math.is_nan(grad)).numpy():
            print("Inf or NaN detected in actor gradients")

episode_rewards = []
epsilon = config.EPSILON_START

for episode in range(config.EPISODES):
    state, info = env.reset()
    state = state / 255.0
    done = False
    episode_reward = 0
    steps = 0
    info ={'lives': 4, 'episode_frame_number': 2, 'frame_number': 2}
    frames = []

    while not done and info.get("lives") >0: #steps < config.MAX_STEPS_PER_EPISODE and info.get("lives") >= 0:
        action = take_action(state, epsilon)

        step_result = env.step(action)
        next_state, reward, done, _, info = step_result[:5]
        next_state = next_state / 255.0

        #next_state, reward, done, _, info = env.step(action)
        #next_state = next_state / 255.0

        memory.push(state, action, next_state, reward, done)
        optimize_model()

        frame = env.render()
        frames.append(frame)

        state = next_state
        episode_reward += reward

        if reward != 0:
            print("step: ", steps, "action: ", action, " reward: ", reward)
            print("Lives: ", info.get("lives"))

        steps += 1

    print(f"Episode: {episode+1}, Reward: {episode_reward}, Epsilon: {epsilon}")

    if episode_reward > best_reward:
      best_reward = episode_reward
      actor_model.save(f"/workspaces/RL_Project/Assault/AC_gifs_plots/best_actor_model", save_format="tf")
      critic_model.save(f"/workspaces/RL_Project/Assault/AC_gifs_plots/best_critic_model", save_format="tf")
      print("New best model saved with reward:", episode_reward)

    episode_rewards.append(episode_reward)

    epsilon = max((epsilon * config.EPSILON_DECAY_RATE), config.EPSILON_END)

    gif_path = f"/workspaces/RL_Project/Assault/AC_gifs_plots/episode_{episode+1}_reward_{episode_reward}.gif"
    
    imageio.mimsave(gif_path, frames, fps=30)

    # Log episode metrics and GIF to wandb
    wandb.log({"episode": episode + 1, "reward": episode_reward, "epsilon": epsilon, "episode_gif": wandb.Video(gif_path, fps=4, format="gif")})

env.close()



In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(10, 5))
plt.plot(episode_rewards)
plt.title("Rewards per Episode")
plt.xlabel("Episode")
plt.ylabel("Reward")
plt.grid(True)
plot_path = "/workspaces/RL_Project/Assault/AC_gifs_plots/rewards_plot.png"
plt.savefig(plot_path)

# Log the plot to WandB
wandb.log({"Training process of Actor-Critic": wandb.Image(plot_path)})

In [ ]:
#TEST THE SAVED MODEL
import gym
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt

# Carga los modelos
actor_model = keras.models.load_model(f"/workspaces/RL_Project/Assault/AC_gifs_plots/best_actor_model")
critic_model = keras.models.load_model(f"/workspaces/RL_Project/Assault/AC_gifs_plots/best_critic_model")

# Crea el entorno
env = gym.make("Assault-v4", render_mode="rgb_array")
n_actions = env.action_space.n

def take_action(state):
    action_probabilities = actor_model.predict(state[np.newaxis, ...])
    return np.random.choice(n_actions, p=np.squeeze(action_probabilities))

# Evalúa el modelo en 10 episodios
rewards_per_episode = []
for episode in range(500):
    state, _ = env.reset()
    state = state / 255.0  # Normaliza los valores de píxeles
    done = False
    episode_reward = 0
    frames = []
    info ={'lives': 4, 'episode_frame_number': 2, 'frame_number': 2}

    while not done:
        action = take_action(state)
        next_state, reward, done, _, info = env.step(action)
        next_state = next_state / 255.0  # Normaliza los valores de píxeles

        frame = env.render()
        frames.append(frame)

        state = next_state
        episode_reward += reward

        if reward != 0:
            print("step: ", steps, "action: ", action, " reward: ", reward)
            print("Lives: ", info.get("lives"))

    rewards_per_episode.append(episode_reward)
    print(f"Episode {episode + 1}: Reward = {episode_reward}")

    gif_path = f"/workspaces/RL_Project/Assault/AC_gifs_plots/test_episode_{episode+1}_reward_{episode_reward}.gif"
    imageio.mimsave(gif_path, frames, fps=30)

    # Log episode metrics and GIF to wandb
    wandb.log({"episode": episode + 1, "reward": episode_reward, "epsilon": epsilon, "episode_gif": wandb.Video(gif_path, fps=4, format="gif")})


env.close()

# Grafica las recompensas por episodio
plt.plot(rewards_per_episode)
plt.xlabel('Episode')
plt.ylabel('Reward')
plt.title('Reward per Episode')
plt.show()

: 

ModuleNotFoundError: No module named 'gym'